In [37]:
from transform import prepare_columns_to_excel, save_dataframe_to_excel_tab

In [38]:
from extract import get_interim_data

df = get_interim_data()
df.head(5)

,president,year,category_(bureaucracy_or_cabinet),agency,party,name_of_authority,total_months_(if_not_the_whole_year),bibliography,conceded_response,partially_conceded_response,denied,others,conceded_and_partially_conceded,total_requests,perc_conceded,perc_partially_conceded,perc_denied,perc_others,perc_concedido_e_parcialmente,category
0,Dilma Rousseff,2015,Burocracia,ABGF - Agencia Brasileira Gestora de Fundos Ga...,None,None,0,None,18,0,0,35,18.0,53.0,0.339623,0.000000,0.0,0.660377,0.339623,neutra
1,Dilma Rousseff,2015,Burocracia,AEB – Agência Espacial Brasileira,None,None,0,None,48,3,0,2,51.0,53.0,0.905660,0.056604,0.0,0.037736,0.962264,neutra
2,Dilma Rousseff,2015,Burocracia,AMAZUL - Amazônia Azul Tecnologias de Defesa S.A.,None,None,0,None,10,3,0,1,10.0,11.0,0.909091,0.000000,0.0,0.090909,0.909091,neutra
3,Dilma Rousseff,2015,Burocracia,AN – Arquivo Nacional,None,None,0,None,147,2,0,7,149.0,156.0,0.942308,0.012821,0.0,0.044872,0.955128,neutra
4,Dilma Rousseff,2015,Burocracia,ANA – Agência Nacional de Águas,None,None,0,None,318,1,0,1,319.0,320.0,0.993750,0.003125,0.0,0.003125,0.996875,neutra


In [41]:
# Selecting only cabinet
ministerios_condition = df['category_(bureaucracy_or_cabinet)'] == "Ministério"
df_ministerios = df[ministerios_condition]# Second version of table

# Columns for the desired table 
desired_cols = [
    'year',
    'agency',
    "perc_concedido_e_parcialmente",
]
df_ministerios_perc_conc_parc = df_ministerios[desired_cols]

# Pivot table to aggregate multiple values
pivoted_df = df_ministerios_perc_conc_parc.pivot_table(index='agency', columns='year', values='perc_concedido_e_parcialmente', aggfunc='mean')

# Changing column names to str
pivoted_df.columns = pivoted_df.columns.astype(str)

# Flattening df to remove multi-level columns
flatenned_df = pivoted_df.reset_index()

# Changing column names to match excel
flatenned_df_to_excel = prepare_columns_to_excel(flatenned_df)

# Checking
flatenned_df_to_excel.head(5)

year,Agency,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,AGU – Advocacia-Geral da União,0.511864,0.471535,0.515152,0.454340,0.582129,0.508238,0.714286,0.593391,0.805255,0.818182
1,CC-PR – Casa Civil da Presidência da República,0.841880,0.770484,0.641379,0.634328,0.809976,0.685714,0.555556,0.577381,0.647639,0.701126
2,CGU – Controladoria-Geral da União,0.684647,0.741766,0.741969,0.717949,0.719971,0.713474,0.699789,0.672182,0.736473,0.726799
3,GSI-PR – Gabinete de Segurança Institucional d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.326772,0.384615
4,MAPA - Ministério da Agricultura e Pecuária,0.860215,0.839797,0.883135,0.872321,0.857547,0.689548,0.695215,0.683855,0.784294,0.766883


Dataset with the party for each agency in each year

In [42]:
# Selecting desired columns
desired_cols = [
    'year',
    'agency',
    "party"
]
df_ministerios_perc_conc_parc = df_ministerios[desired_cols]

# Removing agencies with no parties
df_ministerios_perc_conc_parc = df_ministerios_perc_conc_parc[df_ministerios_perc_conc_parc['party'] != 'no party']
df_ministerios_perc_conc_parc = df_ministerios_perc_conc_parc[df_ministerios_perc_conc_parc['party'].astype(str) != 'None']

# Preparing to excel
df_ministerios_perc_conc_parc_to_excel = prepare_columns_to_excel(df_ministerios_perc_conc_parc)

# Checking
df_ministerios_perc_conc_parc_to_excel.head(5)

,Year,Agency,Party
1074,2017,CC-PR – Casa Civil da Presidência da República,MDB
1077,2017,MAPA - Ministério da Agricultura e Pecuária,PP
1078,2017,MCID - Ministério das Cidades,PSDB
1079,2017,MCID - Ministério das Cidades,PP
1080,2017,"MCTI – Ministério da Ciência, Tecnologia, Inov...",PSD
